In [1]:
from runner import Runner
from model_lgb import ModelLGB
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
plt.rcParams["font.size"] = 18

In [5]:
prms = {
    # lightGBM
    'num_round': 10000,
    'early_stopping_rounds': 10,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metrics': 'huber',
    'num_leaves': 30,
    'learning_rate': 0.3,
    'verbose': 0,
    
    # target
    'target': 'target',
    
    # 街・年
    'cities': ['tokyo'],
    'train_years': [i for i in range(2008, 2017)],
    'valid_years': [2017],
    'test_years': [2018],
    
    # cv
    'cv': 'manual',
}
features = [
    'target','最高気温', '平均気温', '最低気温', '平均湿度',
    '平均現地気圧', '平均蒸気圧', '平均雲量', '平均風速', '日照時間'
]

In [6]:
runner = Runner('test', ModelLGB, features, prms)
runner.run()

Index(['target', '最高気温', '平均気温', '最低気温', '平均湿度', '平均現地気圧', '平均蒸気圧', '平均雲量',
       '平均風速', '日照時間'],
      dtype='object')
target
Index(['target', '最高気温', '平均気温', '最低気温', '平均湿度', '平均現地気圧', '平均蒸気圧', '平均雲量',
       '平均風速', '日照時間'],
      dtype='object')
target
<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 1253 to 1405
Data columns (total 10 columns):
target    153 non-null float64
最高気温      153 non-null float64
平均気温      153 non-null float64
最低気温      153 non-null float64
平均湿度      153 non-null int64
平均現地気圧    153 non-null float64
平均蒸気圧     153 non-null float64
平均雲量      153 non-null float64
平均風速      153 non-null float64
日照時間      153 non-null float64
dtypes: float64(9), int64(1)
memory usage: 13.1+ KB
Index(['target', '最高気温', '平均気温', '最低気温', '平均湿度', '平均現地気圧', '平均蒸気圧', '平均雲量',
       '平均風速', '日照時間'],
      dtype='object')
target


23.032661689519653

In [ ]:
def get_all_multipy_names(multipy_num, feature_names):
    def multipy(target_num, require_least_num, names):
        if target_num == multipy_num:
            name = names[0]
            for i in range(1, multipy_num):
                name += '_x_' + names[i]
            return name
        for i in range(require_least_num, len(feature_names)):
            if feature_names[i] == 'target':
                continue
            result = multipy(target_num + 1, i + 1, names + [feature_names[i]])
            if result is not None:
                all_feature_names.append(result)
    all_feature_names = []
    multipy(0, 0, [])
    return all_feature_names

In [ ]:
feature_names = features.copy()
for i in range(2, 7):
    names = get_all_multipy_names(i, feature_names)
    features += names
features += ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

In [ ]:
features

In [ ]:
dataset = Dataset(features)
#dataset.add_past_day_data([i for i in range(1, 4)])

runner = Runner('test', ModelLGBM, dataset, train_years, test_years, prms)

runner.run_train_cv()
runner.run_predict_cv()

%matplotlib inline
pred, correct,_ = runner.get_predict_cv()
plt.figure(figsize=(30, 30))
plt.plot(pred, color='r')

plt.plot(correct.values, color='black')
plt.show()

In [ ]:
# 特徴量の重要度をプロットする
model = ModelLGBM('test-3', features)
model.load_model()
plt.figure(figsize=(200, 200))
lgb.plot_importance(model.model, figsize=(3, 100))
plt.savefig('data.png')

In [ ]:
features = [
    '最高気温',
    '最高気温_x_平均気温',
    '最高気温_x_平均気温_x_平均蒸気圧',
    'target'
]

dataset = Dataset(features)

runner = Runner('test', ModelLGBM, dataset, train_years, test_years, prms)

runner.run_train_cv()
runner.run_predict_cv()

%matplotlib inline
pred, correct,_ = runner.get_predict_cv()
plt.figure(figsize=(30, 30))
plt.plot(pred, color='r')

plt.plot(correct.values, color='black')
plt.show()

In [ ]:
# 特徴量の重要度をプロットする
model = ModelLGBM('test-3', features)
model.load_model()
lgb.plot_importance(model.model, figsize=(30, 20))
plt.savefig('data.png')